# Parsing quarterly FEC filings for summary statistics of key races

A series of python manipulations (with a little bash) for extracting and organizing “medium” amounts of FEC data.

## Prerequisites

* [fec loader](https://github.com/PublicI/fec-loader)
* [PostgreSQL](https://www.postgresql.org/)
* Python things
    * pandas
    * numpy
    * datetime
    * subprocess
    
---

## Setup

*Import our libraries*

In [ ]:
import os
import pandas as pd
import numpy as np
import datetime
import subprocess
import psycopg2

## Source data

*Before beginning we need to download our source data from* https://www.fec.gov/data/filings/?data_type=efiling.

*Name this file* `filings.csv` *and place it in the* `data/source` *folder.*

*Now we set a variable called `file` to that path:*

In [ ]:
file = '../data/source/filings.csv'



## Variables

*Here we'll list some other things that we can change depending on what time frame, report types, and candidates we're interested in.*


**Report types:**  
*Report types represent the different forms committees are required to submit to the FEC at different times. While it's tempting to just grab the quarterly reports (Q1, Q2, Q3, and YE) those will miss the full picture in the event of primaries, runoffs, etc., because campaigns file before and after those events.*

*Find a description of the different reporte types here:* https://www.fec.gov/campaign-finance-data/report-type-code-descriptions/

In [ ]:
report_types = ['12P','12C','12R','Q2','12S','30R']

<br>**Committees:**  
*What committees are you trying to gather data about? Include them in this list. Committees are given a unique identifier that almost always starts with `C00`. You can find that identifier by visiting the committee's page and looking at the URL.*

*For example, the URL `https://www.fec.gov/data/committee/C00012229/` corresponds to the committee `ALASKANS FOR DON YOUNG` and we'll extract the last portion of that URL `C00012229` for our list.*

In [ ]:
committees = ['C00710822','C00570994','C00665711','C00607515','C00012229','C00640623','C00701672','C00726943','C00551275','C00696526','C00666040','C00582866','C00582890','C00694091','C00657155','C00722694','C00651042','C00736918','C00708149','C00698084','C00698373','C00710582','C00734087','C00725614','C00649897','C00655134','C00699249','C00723197','C00735175','C00540617','C00639146','C00681544','C00648956','C00499392','C00708479','C00370056','C00725101','C00701102','C00650648','C00665638','C00636571','C00700559','C00633982','C00704981','C00634253','C00666859','C00635888','C00721332','C00716720','C00492454','C00707638','C00728238','C00637363','C00717868','C00620443','C00740852','C00723346','C00726000','C00726596','C00590067','C00667808','C00708263','C00724062','C00742973','C00708222','C00713909','C00718650','C00676668','C00742247','C00713222','C00412759','C00721290','C00743286','C00632257','C00721803','C00652065','C00716225','C00735985','C00672311','C00671859','C00715474','C00722801','C00718940','C00689596','C00718866','C00721480','C00726075','C00733030','C00736876','C00737700','C00738203','C00741058','C00738716','C00547570','C00720508','C00726695','C00729608','C00736983','C00672295','C00633362','C00649376','C00706747','C00708164','C00546788','C00637074','C00706267','C00706457','C00558825','C00646844','C00545616','C00650622','C00693663','C00648493','C00712794','C00649483','C00521948','C00652719','C00696872','C00498568','C00716498','C00711887','C00737767','C00712455','C00723122','C00576173','C00710939','C00711325','C00735464','C00743153','C00732792','C00660050','C00718346','C00748103','C00670034','C00714089','C00717876','C00721027','C00741132','C00714279','C00467571','C00196774','C00512970','C00739177','C00638148','C00709899','C00314575','C00653816','C00722108','C00722454','C00437889','C00651208','C00711317','C00692830','C00704270','C00710814','C00710962','C00722769','C00701599','C00727826','C00706770','C00200584','C00722611','C00650150','C00721118','C00722587','C00726687','C00726992','C00729186','C00638650','C00656033','C00691535','C00714006','C00714519','C00725259','C00735910','C00649327','C00722074','C00703116','C00741025','C00550707','C00707190','C00575209','C00719039','C00731281','C00732925','C00640714','C00720730','C00713461','C00253187','C00679464','C00738385','C00610071','C00664862','C00699488','C00717959','C00718676','C00729558','C00495846','C00741611','C00491357','C00701748','C00548289','C00709410','C00545772','C00729277','C00731885','C00662577','C00732651','C00732800','C00504522','C00731919','C00699660','C00639310','C00575167','C00660464','C00733543','C00734491','C00735001','C00732545','C00661868','C00648220','C00715466','C00573949','C00706275','C00656686','C00703058','C00640003','C00711572','C00666149','C00655571','C00655613','C00717322','C00668228','C00694711','C00711564','C00552547','C00706549','C00729954','C00652248','C00694778','C00512426','C00701433','C00633859','C00737650','C00648725','C00632828','C00655183','C00556365','C00666453','C00706788','C00711630','C00301838','C00713743','C00373001','C00724047','C00662650','C00648915','C00703843','C00704064','C00215905','C00715854','C00713693','C00607416','C00658567','C00722892','C00509968','C00735688','C00710533','C00510164','C00722314','C00474189','C00657411','C00724914','C00696153','C00458828','C00650507','C00710103','C00649350','C00369033','C00730408','C00660795','C00640045','C00701003','C00662668','C00392688','C00713297','C00662767','C00662874','C00730150','C00652297','C00705079','C00706614','C00702225','C00714865','C00710475','C00371203','C00637868','C00715235','C00658633','C00725853','C00664375','C00733394','C00714964','C00721308','C00649913','C00704643','C00714204','C00715482','C00721126','C00727594','C00729335','C00638023','C00721142','C00660472','C00700930','C00472704','C00709642','C00652628','C00737106','C00700161','C00720870','C00295931','C00312017','C00713982','C00716258','C00637686','C00724641','C00730580','C00736108','C00737924','C00742007']

<br>**Coverage dates:**  
*These variables target the columns `coverage_start_date` and `coverage_end_date` in our `filings.csv` file. Generally, these are going to be the dates for the most recent filing quarter. Here the format `YYYY-MM-DD` is important.*

In [ ]:
cover_start = '2020-04-01'
cover_end = '2020-06-30'

<br>**Database name:**  
*The next two variables use Python's [datetime library](https://docs.python.org/3/library/datetime.html) to name our database. We use a datetime variable to name our database because it's often useful to run this script a number of times right around the quarterly filing deadlines. Say, once late the evening of the deadline and once again the next morning to catch any late filers. This way we have a unique name for our database that is based on when we ran this script and we don't have to manually change it every time we want to run it.*

*The format `%b%d%Y%H%M` corresponds to, jan3120201645, if we ran this script on January 31, 2020 at 4:45 p.m., for example.*

In [ ]:
d = datetime.datetime.now()
db = d.strftime("%b%d%Y%H%M").lower()

## Slicing and dicing

*This section assigns variables to targeted selections of the broader dataset for later use.*

*First we read the data into the [Pandas library](https://pandas.pydata.org/).*

*Note: `df` is a Pandas convention that stands for dataframe.*

In [ ]:
df = pd.read_csv(file, keep_default_na=False, dtype={'file_number': object, 'form_type': object, 'committee_id': object, 'committee_name': object, 'receipt_date': object, 'load_timestamp': object, 'coverage_start_date': object, 'coverage_end_date': object, 'beginning_image_number': int, 'ending_image_number': int, 'report_type': object, 'amends_file': object, 'amendment_number': int, 'filed_date': object}) #USE IN DEV IPYNB

*The options `keep_default_na=False` and `dtype={blablabla}` refer to [how Pandas handles empty or null data points](https://pandas.pydata.org/pandas-docs/stable/user_guide/missing_data.html) and making sure we tell Pandas what kind of data is in each column.*

*Basically, we want to override the default null settings because substituting `nan` for every empty cell creates major headaches later. And we want to tell pandas when a number is a number for doing math, such as dollars, or a number for identifying something, such as `filing_id`.*

**Only the committees we're interested in:**  
*This creates a variable that limits our data to just the committees listed in our earlier `committees` variable.*

In [ ]:
committees_df = df[df['committee_id'].isin(committees)]

<br>**Only the dates we're interested in:**  
*This creates a variable that further limits our data to just the timeframe we want based on the `cover_start` and `cover_end` variables.*

In [ ]:
dates_df = committees_df[(committees_df['coverage_start_date'] >= cover_start) & (committees_df['coverage_end_date'] <= cover_end)]

<br>**Amended filings:**  
*Amended filings usually supercede previous filings, so we want to take out those previous filings.*

*First we make a list of the amended filings:*

In [ ]:
amendments = list(filter(lambda x: x is not '', dates_df['amends_file']))

*Then we filter them out.*

In [ ]:
amend_filter_df = dates_df[~dates_df['file_number'].isin(amendments)]

<br>**Terminated campaigns:**  
*We're not interested in them ...*
*First we gather them in a list.*

In [ ]:
terminations = amend_filter_df.loc[amend_filter_df['report_type'] == 'TER', 'committee_id'].tolist()

*Then we make a dataframe without them.*

In [ ]:
no_terminations_df = amend_filter_df[~amend_filter_df['committee_id'].isin(terminations)]

<br>**Report types:**  
*Now that we've narrowed down our dataset, it's useful to know what different report types are present, so that we can make sure none are missing from the `report_types` variable.*

*This next line tells us that:*

In [ ]:
for r in no_terminations_df['report_type'].unique(): print(r)

*Once we've checked our `report_types` list and made any necessary changes we can make a dataset filtering for just those reports we're interested in.*

In [ ]:
reports_filter_df = no_terminations_df[no_terminations_df['report_type'].isin(report_types)]

<br>**The FEC files:**<br>
*The next sections requires a list of all the .fec files to plug into [fec-loader](https://github.com/PublicI/fec-loader).*

In [ ]:
files = list(reports_filter_df['file_number'].astype(int))

<br>

**Sanity check**  
*Print some stuff out to see if things pass the sniff test.*

In [ ]:
print('STATS:')
print('\nNumber of selected committees:', len(committees))
print('\nNumber of .fec files:', len(files))
print('\nNumber of terminations:', len(terminations), '\n')
print('Terminated committees: \n')
for i in df['committee_name'][df['committee_id'].isin(terminations)].unique().tolist():
    print(i)
print('\nNumber of amendments:', len(amendments))
print('\nReport types included:', report_types)
print('\nCommittees not accounted for:\n')
for i in committees:
    if i not in terminations + df['committee_id'][df['file_number'].astype(int).isin(files)].tolist():
        print(i)

## Using fec-loader

*fec-loader will take all of the files in our `files` variable and put them into a neat PostgreSQL database.*

*In order to do that we need to run a bash command in Python.*

In [ ]:
subprocess.run(["fec", "init"], env=dict(PGHOST="localhost", PGDATABASE=db, PGUSER="postgres", PGPASSWORD="", **os.environ))

*You can see the output and any errors generated by these commands in the terminal window running this notebook.*

*The above cell is equivalent to running the following in your terminal where `db` would be the datetime stamp established when the sixth cell of this notebook was run (`jan3120201645`, to continue the example from up there):*

```
$ export PGHOST=localhost PGDATABASE=db PGUSER=postgres PGPASSWORD=
$ fec init
```

*To see if it worked use your favorite postgres tool to look for a localhost database with a name of whatever the datetime was when you ran the sixth cell of this notebook.*

*Next we'll use fec-loader to fill the database. (This one can take a little while depending on how many committees you're working with. You can comment out the print statment but I have it in there so I can see that something is still moving. You can also refresh the postgres database to monitor progress.)*

In [ ]:
for f in files:
    subprocess.run([f"curl -s 'https://docquery.fec.gov/dcdev/posted/{f}.fec' | fec convert {f} --format=psql | psql"], shell=True, env=dict(PGHOST="localhost", PGDATABASE=db, PGUSER="postgres", PGPASSWORD="", **os.environ))

*At this point if you're happy using the postgres database you can stop here.*

*If you want to do some more data manipulation with python and export the result, read on.*

## Connecting to postgres

*We'll set up a connection to the local postgres database using a Python library called psycopg2*

*The first line establishes the connection, the second line builds a dataframe based on an SQL query.*


In [ ]:
conn = psycopg2.connect(host="localhost", database=db, user="postgres", password="")

df_pg = pd.read_sql_query('SELECT filing_id, form_type, filer_committee_id_number, committee_name, report_code, coverage_from_date, coverage_through_date, col_a_total_individual_contributions, col_a_individual_contributions_unitemized, col_a_pac_contributions, col_a_candidate_contributions, col_a_candidate_loans, col_a_total_receipts, col_b_total_receipts, col_a_total_disbursements, col_a_cash_on_hand_close FROM fec_campaign_summaries', conn)

**More variables:**  
*We've got two above already but we need to set a couple more to extract the specific information we're looking for.*

*One for all the committees (we'll use `cmts` since committees was taken above) and one for all the committees that have filed their quarterly.*

**Note:** *Change `Q2` here to whatever corresponding quarter you're working on Q1, Q2, Q3 or YE for fourth quarter.*

In [ ]:
cmts = df_pg['filer_committee_id_number'].unique().tolist()

quarterly_filed = df_pg['filer_committee_id_number'][df_pg['report_code'] == 'Q2'].tolist()

## Building a whole new dataframe:
*Next were going to set up some empty lists and a dictionary to build a new dataframe by doing some grouping and math on the data in `df_pg`*

*First the empty lists, which are based on the column names in `df_pg` and will make up the columns of our new dataframe. Plus we'll add `has_quarterly`, which we'll use to keep track of whether or not the quarterly report has been filed:*

In [ ]:
filer_committee_id_number = []
col_a_total_individual_contributions = []
col_a_individual_contributions_unitemized = []
col_a_pac_contributions = []
col_a_candidate_contributions = []
col_a_candidate_loans = []
col_a_total_receipts = []
col_b_total_receipts = []
col_a_total_disbursements = []
col_a_cash_on_hand_close = []
has_quarterly = []

*Next we make a dictionary of those lists, which will be the basis for the data in our new dataframe that we'll call `df_pg2`. Later we'll make a loop that populates the list and therefore the dictionary and therefore the dataframe.*

In [ ]:
d = {
        'filer_committee_id_number': filer_committee_id_number,
        'col_a_total_individual_contributions': col_a_total_individual_contributions,
        'col_a_individual_contributions_unitemized': col_a_individual_contributions_unitemized,
        'col_a_pac_contributions': col_a_pac_contributions,
        'col_a_candidate_contributions': col_a_candidate_contributions,
        'col_a_candidate_loans': col_a_candidate_loans,
        'col_a_total_receipts': col_a_total_receipts,
        'col_b_total_receipts': col_b_total_receipts,
        'col_a_total_disbursements': col_a_total_disbursements,
        'col_a_cash_on_hand_close': col_a_cash_on_hand_close,
        'has_quarterly': has_quarterly
}

## Populating `d` lists with a loop

*This for loop with an if statement looks big but it's really just one pattern for each column and they're mostly the same except for `has_quarterly`, `col_a_cash_on_hand_close`, and `col_b_total_receipts`.*

In [ ]:
for cmt in cmts:
    if cmt in quarterly_filed:
        d_filer_committee_id_number = cmt
        d_col_a_total_individual_contributions = df_pg['col_a_total_individual_contributions'][df_pg['filer_committee_id_number'] == cmt].sum()
        d_col_a_individual_contributions_unitemized = df_pg['col_a_individual_contributions_unitemized'][df_pg['filer_committee_id_number'] == cmt].sum()
        d_col_a_pac_contributions = df_pg['col_a_pac_contributions'][df_pg['filer_committee_id_number'] == cmt].sum()
        d_col_a_candidate_contributions = df_pg['col_a_candidate_contributions'][df_pg['filer_committee_id_number'] == cmt].sum()
        d_col_a_candidate_loans = df_pg['col_a_candidate_loans'][df_pg['filer_committee_id_number'] == cmt].sum()
        d_col_a_total_receipts = df_pg['col_a_total_receipts'][df_pg['filer_committee_id_number'] == cmt].sum()
        d_col_b_total_receipts = df_pg['col_b_total_receipts'][df_pg['filer_committee_id_number'] == cmt][df_pg['coverage_through_date'] == df_pg['coverage_through_date'][df_pg['filer_committee_id_number'] == cmt].max()].item()
        d_col_a_total_disbursements = df_pg['col_a_total_disbursements'][df_pg['filer_committee_id_number'] == cmt].sum()
        d_col_a_cash_on_hand_close = df_pg['col_a_cash_on_hand_close'][df_pg['filer_committee_id_number'] == cmt][df_pg['coverage_through_date'] == df_pg['coverage_through_date'][df_pg['filer_committee_id_number'] == cmt].max()].item()
        d_has_quarterly = 'YES'
        
        filer_committee_id_number.append(d_filer_committee_id_number)
        col_a_total_individual_contributions.append(d_col_a_total_individual_contributions)
        col_a_individual_contributions_unitemized.append(d_col_a_individual_contributions_unitemized)
        col_a_pac_contributions.append(d_col_a_pac_contributions)
        col_a_candidate_contributions.append(d_col_a_candidate_contributions)
        col_a_candidate_loans.append(d_col_a_candidate_loans)
        col_a_total_receipts.append(d_col_a_total_receipts)
        col_b_total_receipts.append(d_col_b_total_receipts)
        col_a_total_disbursements.append(d_col_a_total_disbursements)
        col_a_cash_on_hand_close.append(d_col_a_cash_on_hand_close)
        has_quarterly.append(d_has_quarterly)
        
    else:
        d_filer_committee_id_number = cmt
        d_col_a_total_individual_contributions = df_pg['col_a_total_individual_contributions'][df_pg['filer_committee_id_number'] == cmt].sum()
        d_col_a_individual_contributions_unitemized = df_pg['col_a_individual_contributions_unitemized'][df_pg['filer_committee_id_number'] == cmt].sum()
        d_col_a_pac_contributions = df_pg['col_a_pac_contributions'][df_pg['filer_committee_id_number'] == cmt].sum()
        d_col_a_candidate_contributions = df_pg['col_a_candidate_contributions'][df_pg['filer_committee_id_number'] == cmt].sum()
        d_col_a_candidate_loans = df_pg['col_a_candidate_loans'][df_pg['filer_committee_id_number'] == cmt].sum()
        d_col_a_total_receipts = df_pg['col_a_total_receipts'][df_pg['filer_committee_id_number'] == cmt].sum()
        d_col_b_total_receipts = df_pg['col_b_total_receipts'][df_pg['filer_committee_id_number'] == cmt][df_pg['coverage_through_date'] == df_pg['coverage_through_date'][df_pg['filer_committee_id_number'] == cmt].max()].item()
        d_col_a_total_disbursements = df_pg['col_a_total_disbursements'][df_pg['filer_committee_id_number'] == cmt].sum()
        d_col_a_cash_on_hand_close = df_pg['col_a_cash_on_hand_close'][df_pg['filer_committee_id_number'] == cmt][df_pg['coverage_through_date'] == df_pg['coverage_through_date'][df_pg['filer_committee_id_number'] == cmt].max()].item()
        d_has_quarterly = 'NO'
        
        filer_committee_id_number.append(d_filer_committee_id_number)
        col_a_total_individual_contributions.append(d_col_a_total_individual_contributions)
        col_a_individual_contributions_unitemized.append(d_col_a_individual_contributions_unitemized)
        col_a_pac_contributions.append(d_col_a_pac_contributions)
        col_a_candidate_contributions.append(d_col_a_candidate_contributions)
        col_a_candidate_loans.append(d_col_a_candidate_loans)
        col_a_total_receipts.append(d_col_a_total_receipts)
        col_b_total_receipts.append(d_col_b_total_receipts)
        col_a_total_disbursements.append(d_col_a_total_disbursements)
        col_a_cash_on_hand_close.append(d_col_a_cash_on_hand_close)
        has_quarterly.append(d_has_quarterly)

# Put it all in a dataframe and export to .csv

In [ ]:
df_pg2 = pd.DataFrame(data = d)
df_pg2.round(2) #Rounds to two decimal places

In [ ]:
df_pg2.round(2).to_csv(r'../data/processed/' + db +'.csv', index = False) #Exports to CSV

# Run some sanity checks

In [ ]:
# GET A LIST OF THE COMMITTEES WITH AMENDMENTS

for id in df_pg['filer_committee_id_number'].unique():
    if id in dates_df['committee_id'][dates_df['amends_file'] != ''].to_list():
        print(df_pg['committee_name'][df_pg['filer_committee_id_number'] == id].to_string(), '—', id)

In [ ]:
# LOOK TO SEE IF ANY COMMITTEES FILED DOUBLE FORMS

for id in df_pg['filer_committee_id_number'].unique():
    group = df_pg[df_pg['filer_committee_id_number'] == id]
    ls = group['report_code'].to_list()
    if len(ls) != len(set(ls)):
        print(f'FLAG! — {id}', df_pg['committee_name'][df_pg['filer_committee_id_number'] == id])
    else:
        print(id, 'OK!')

In [ ]:
# LOOK TO SEE WHICH COMMITTEES ARE NOT IN THE DATABASE

for id in committees:
    if id not in df_pg['filer_committee_id_number'].unique():
        print(id)